<a href="https://colab.research.google.com/github/doyeo-n/-Capstone-8_code/blob/main/%5B8%5DLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def splitdataset(data, timestep):
  from array import array
  data = data.set_index(index_col)
  sha = data.shape
  a = round(sha[0]*0.3)
  b = len(data[:-a])
  c = b%7
  d = len(data[-a : -6])
  e = d%7

  train, test = data[c:-a], data[-a+e:-(timestep-1)] 
  return train, test

def split_dataset(data, timestep):
  train, test = splitdataset(data, timestep = 7)
  train = array(split(train, int(len(train)/timestep)))
  test = array(split(test, len(test)/timestep))
  return train, test
 
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, -1])
		# move along one time step
		in_start += 1
	return array(X), array(y)

# train the model
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)

	# define parameters
	verbose, epochs, batch_size = verbose_a, epochs_a, batch_size_a
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
 
	# define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(200, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(100, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
                         
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat


def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
 
	# history is a list of weekly data
	history = [x for x in train]

	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	#score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	score, scores = evaluate_forecasts(test[:, :, -1], predictions)
	return  model, score, scores, predictions

In [ ]:
#result
def rmse_plot():
  days = ['D+1', 'D+2', 'D+3', 'D+4', 'D+5', 'D+6', 'D+7']
  pyplot.plot(days, scores, marker='o', label='lstm')
  pyplot.show()

def comparison():
  #실제값
  da_test = test[-1,:,-1]
  #예측값
  da_pred = predictions[-1]

  da_test = pd.DataFrame(da_test)
  da_pred = pd.DataFrame(da_pred)
  data_f = concat([da_test, da_pred], axis=1)
  list_a = ['d+1', 'd+2', 'd+3', 'd+4', 'd+5', 'd+6', "d+7"]
  list_a = pd.DataFrame(list_a)
  data_f = concat([data_f, list_a], axis=1)
  data_f.columns = ['test', 'pred', 'index']
  data_f = data_f.set_index('index')
  data_f
  return data_f

def plot_3(imputed_data):
  p = np.ravel(predictions)
  t = np.ravel(test[:,:,-1])
  data_kalman= imputed_data
  data_kalman = data_kalman.set_index('date')

  original['masking'] = 1
  mask = original['masking']
  mask.columns = ['masking']
  #df_new['date-like_column'] = pd.to_datetime(df_new['date-like-column']
  data_mask = pd.merge(imputed_data, mask,left_index=True, right_index=True, how='left')
  data_mask[(data_mask['masking']=='NaN')]=0

  data_mask['masking'] = data_mask['masking'].fillna(0)

  data_mask = data_mask.iloc[-916:, :]
  data_mask['log_cyan'][(data_mask['masking']==0)] = np.nan
  rea_y = np.array(data_mask['log_cyan'])
  print(len(rea_y))

  #실제값/예측값
  pyplot.figure(figsize=(15, 10))
  #pyplot.plot(inv_y, 'ko')
  pyplot.plot(p, 'r.-')
  pyplot.plot(rea_y, 'ko')
  pyplot.legend(["Pred", "Obs"], bbox_to_anchor = (1,1) )
  pyplot.show()

  #임퓨테이션한 값/ 예측값
  pyplot.figure(figsize=(15, 10))
  pyplot.plot(t)
  pyplot.plot(p)

  pyplot.legend(["Obs", "Pred"], bbox_to_anchor = (1,1) )
  pyplot.show()

  #one-to-one plot
  plt.figure(figsize=(5,5))
  plt.scatter(t, p, color = 'red')
  lims = [np.min([0, 15]), np.max([0, 15])]
  plt.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
  plt.xlim(lims)
  plt.ylim(lims)
  plt.xlabel("real")
  plt.ylabel("predicted")
  plt.show()

  import gc
  gc.collect()
  """
  print(inv_yhat.shape)
  print(inv_y.shape)
  """
def classification(predictions, test):
  pred = predictions.flatten()
  true = test[:,:,-1].flatten()
  pred = pd.DataFrame(pred)
  true = pd.DataFrame(true)
  pred['class'] = np.nan
  true['class'] = np.nan

  math.log(784)
  a = math.log(1000)
  b = math.log(10000)
  c = math.log(1000000)

  for i in range (len(pred)):
    if pred.iloc[i,0] < a :
      pred.iloc[i,1] = 0
    elif a <= pred.iloc[i,0] <b :
      pred.iloc[i,1] = 1
    elif b<= pred.iloc[i,0] < c:
      pred.iloc[i,1] = 2
    elif c<= pred.iloc[i,0]:
      pred.iloc[i,1] = 3
    else:
      pred.iloc[i,1] = 999


  for i in range (len(true)):
    if true.iloc[i,0] < a :
      true.iloc[i,1] = 0
    elif a <= true.iloc[i,0] <b :
      true.iloc[i,1] = 1
    elif b<= true.iloc[i,0] < c:
      true.iloc[i,1] = 2
    elif c<= true.iloc[i,0]:
      true.iloc[i,1] = 3
    else:
      true.iloc[i,1] = 999

  from sklearn.metrics import accuracy_score
  acc_test = accuracy_score(true['class'], pred['class'])
  print(acc_test)
  from sklearn.metrics import classification_report, confusion_matrix
  print(classification_report(true['class'], pred['class']))
  print(confusion_matrix(true['class'], pred['class']))

#test 중 실제값이 있는 데이터에 한해 분류 성능 평가
def test_classification(predcitions, test):
  df_m = eutrophication.iloc[1:,:]
  data=df_m
  train, test_m = data[1:-916], data[-916:-6]
  train = array(split(train, int(len(train)/7)))
  test_m = array(split(test_m, len(test_m)/7))
  print(train.shape, test_m.shape)

  print(test_m[:,:,-1].shape, predictions.shape)

  pred = predictions.flatten()
  true_m = test_m[:,:,-1].flatten()
  true_m = pd.DataFrame(true_m)
  pred = pd.DataFrame(pred)

  da = pd.concat([true_m, pred], axis=1)

  da.columns = ['true', 'pred']
  da = da.dropna()

  true_m = da['true']
  pred = da['pred']
  pred = pd.DataFrame(pred)
  true_m = pd.DataFrame(true_m)
  pred['class'] = 'nan'
  true_m['class']='nan'

  import math
  math.log(784)
  a = math.log(1000)
  b = math.log(10000)
  c = math.log(1000000)

  pred = pred.reset_index(drop=True, inplace=False)
  true_m = true_m.reset_index(drop=True, inplace=False)

  for i in range (len(pred)):
    if pred.iloc[i,0] < a :
      pred.iloc[i,1] = 0
    elif a <= pred.iloc[i,0] <b :
      pred.iloc[i,1] = 1
    elif b<= pred.iloc[i,0] < c:
      pred.iloc[i,1] = 2
    elif c<= pred.iloc[i,0]:
      pred.iloc[i,1] = 3
    else:
      pred.iloc[i,1] = 999


  for i in range (len(true_m)):
    if true_m.iloc[i,0] < a :
      true_m.iloc[i,1] = 0
    elif a <= true_m.iloc[i,0] <b :
      true_m.iloc[i,1] = 1
    elif b<= true_m.iloc[i,0] < c:
      true_m.iloc[i,1] = 2
    elif c<= true_m.iloc[i,0]:
      true_m.iloc[i,1] = 3
    else:
      true_m.iloc[i,1] = 999

  from sklearn.metrics import accuracy_score
  from sklearn.metrics import classification_report, confusion_matrix

  true_new = true_m.iloc[:,1].tolist()
  pred_new = pred.iloc[:,1].tolist()

  acc_test = accuracy_score(true_new, pred_new)
  print(acc_test)

  print(classification_report(true_new, pred_new))
  print(confusion_matrix(true_new, pred_new))


In [ ]:
#결정사항
verbose_a, epochs_a, batch_size_a=1, 10, 1
timestep = 7

#LSTM 모델링
train, test = split_dataset(imputed_data, timestep)
model, score, scores, predictions = evaluate_model(train, test, n_input)

#LSTM 모델링 결과 확인
print(f'test 전체 rmse : {score}, 마지막 7일에 대한 각각의 rmse : {score}')
rmse_plot()

comparison()

plot_3(imputed_data)

print('test 데이터에 대한 분류 혼동행렬')
classification(predictions, test)
print('test 데이터 중 실제 존재하는 값에 대한 분류 혼동행렬')
test_classification(predictions, test)